In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.optim as optim
from tqdm import tqdm

In [2]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [3]:
class VehicleDataset(Dataset):
    
    def __init__(self, path):
        data = np.load(path)
        self.images = data["images"]
        self.labels = data["labels"]
        print("Images shape:", self.images.shape)
        print("Labels shape:", self.labels.shape)
        self.__combinelabels__()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)

        #mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        #std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        #image = (image - mean) / std
        
        return image, label
    def __combinelabels__(self):
        for idx, label in enumerate(self.labels):
            if label == 1:
                self.labels[idx] = 1
            if label >= 2 and label <= 7:
                self.labels[idx] = 2
            if label >= 8 and label <= 11:
                self.labels[idx] = 3
            if label >= 12 and label <= 25:
                self.labels[idx] = 4
            if label >= 26 and label <= 38:
               self.labels[idx] = 5
            if label >= 39 and label <= 44:
                self.labels[idx] = 6
            if label >= 45 and label <= 50:
                self.labels[idx] = 7
            if label >= 51 and label <= 53:
                self.labels[idx] = 8
            if label >= 54 and label <= 75:
                self.labels[idx] = 9
            if label >= 76 and label <= 81:
                self.labels[idx] = 10
            if label == 82:
                self.labels[idx] = 11
            if label >= 83 and label <= 97:
                self.labels[idx] = 12
            if label == 98:
                self.labels[idx] = 13
            if label >= 99 and label <= 100:
                self.labels[idx] = 14
            if label >= 101 and label <= 104:
                self.labels[idx] = 15
            if label == 105:
                self.labels[idx] = 16
            if label >= 106 and label <= 117:
                self.labels[idx] = 17
            if label >= 118 and label <= 122:
                self.labels[idx] = 18
            if label == 123:
                self.labels[idx] = 19
            if label >= 124 and label <= 125:
                self.labels[idx] = 20
            if label >= 126 and label <= 129:
                self.labels[idx] = 21
            if label >= 130 and label <= 140:
                self.labels[idx] = 22
            if label >= 141 and label <= 142:
                self.labels[idx] = 23
            if label == 143:
                self.labels[idx] = 24
            if label == 144:
                self.labels[idx] = 25
            if label >= 145 and label <= 149:
                self.labels[idx] = 26
            if label >= 150 and label <= 153:
                self.labels[idx] = 27
            if label >= 154 and label <= 155:
                self.labels[idx] = 28
            if label == 156:
                self.labels[idx] = 29
            if label == 157:
                self.labels[idx] = 30
            if label == 158:
                self.labels[idx] = 31
            if label == 159:
                self.labels[idx] = 32
            if label == 160:
                self.labels[idx] = 33
            if label >= 161 and label <= 166:
                self.labels[idx] = 34
            if label == 167:
                self.labels[idx] = 35
            if label >= 168 and label <= 171:
                self.labels[idx] = 36
            if label == 172:
                self.labels[idx] = 37
            if label == 173:
                self.labels[idx] = 38
            if label == 174:
                self.labels[idx] = 39
            if label >= 175 and label <= 177:
                self.labels[idx] = 40
            if label == 178:
                self.labels[idx] = 41
            if label >= 179 and label <= 180:
                self.labels[idx] = 42
            if label >= 181 and label <= 184:
                self.labels[idx] = 43
            if label == 185:
                self.labels[idx] = 44
            if label >= 186 and label <= 189:
                self.labels[idx] = 45
            if label >= 190 and label <= 192:
                self.labels[idx] = 46
            if label >= 193 and label <= 195:
                self.labels[idx] = 47
            if label == 196:
                self.labels[idx] = 48

In [16]:
dataset = VehicleDataset('../dataset/stanford_cars_dataset.npz')

batch_size = 32
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = int(len(dataset) - train_size - val_size)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Images shape: (8144, 112, 112, 3)
Labels shape: (8144, 1)


In [5]:
def train_model(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    num_epochs = 30
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
    
        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels[:,0].to(device) - 1

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        scheduler.step()
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")
    
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels[:,0].to(device) - 1
            
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f'Validation Acc: {val_acc:.4f}')

In [6]:
def test_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    all_preds = []
    all_labels = []
    rmse = 0.0
    mae = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels[:,0].to(device) - 1 
            
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
            # print(labels.shape)
            # rmse += np.sqrt(np.mean((preds - labels) ** 2)) * images.size(0)
            # mae = torch.abs(preds - labels).sum()

    accuracy = correct / total

    print(f'Test Accuracy: {accuracy:.4f}')
    
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    mae = torch.mean(torch.abs(all_preds.float() - all_labels.float())).item()
    rmse = torch.sqrt(torch.mean((all_preds.float() - all_labels.float()) ** 2)).item()

    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    
    return accuracy, all_preds, all_labels

In [19]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        return torch.cat([x, out], 1) 

class Block(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate=32):
        super(Block, self).__init__()
        layers = []
        for i in range(num_layers):
            layers.append(DenseLayer(in_channels + (i * growth_rate), growth_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.avgpool = nn.AvgPool2d(2)

    def forward(self, x):
        x = self.conv(self.relu(self.bn(x)))
        return self.avgpool(x)

class DenseNet(nn.Module):
    def __init__(self, num_classes=96, growth_rate=32):
        super(DenseNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )

        num_features = 64
        config_list = [6, 12, 24, 16] 

        self.dense_blocks = nn.ModuleList()
        self.transitions = nn.ModuleList()

        for i, num_layers in enumerate(config_list):
            block = Block(num_layers, num_features, growth_rate)
            self.dense_blocks.append(block)
            num_features += num_layers * growth_rate

            if i != len(config_list) - 1:
                transition = Transition(num_features, num_features // 2)
                self.transitions.append(transition)
                num_features = num_features // 2

        self.bn = nn.BatchNorm2d(num_features)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(num_features, 48)

    def forward(self, x):
        x = self.features(x)
        for i in range(len(self.dense_blocks)):
            x = self.dense_blocks[i](x)
            if i < len(self.transitions):
                x = self.transitions[i](x)
        x = self.avgpool(self.relu(self.bn(x)))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [20]:
model = DenseNet().to(device)
train_model(model)

100%|██████████| 179/179 [00:13<00:00, 13.49it/s]


Epoch 1/30 - Loss: 3.3976 - Accuracy: 0.1198
Validation Acc: 0.1179


100%|██████████| 179/179 [00:13<00:00, 13.41it/s]


Epoch 2/30 - Loss: 3.1424 - Accuracy: 0.1593
Validation Acc: 0.1296


100%|██████████| 179/179 [00:13<00:00, 13.11it/s]


...

Epoch 28/30 - Loss: 0.0826 - Accuracy: 1.0000
Validation Acc: 0.3225


100%|██████████| 179/179 [00:15<00:00, 11.61it/s]


Epoch 29/30 - Loss: 0.0811 - Accuracy: 0.9993
Validation Acc: 0.3194


100%|██████████| 179/179 [00:14<00:00, 12.40it/s]


Epoch 30/30 - Loss: 0.0797 - Accuracy: 1.0000
Validation Acc: 0.3213


In [21]:
torch.save(model.state_dict(), "DenseNet121")

In [22]:
test_model(model, test_loader, device)

Test Accuracy: 0.3333
MAE: 10.2071
RMSE: 15.9313


(0.3333333333333333,
 tensor([ 8, 35, 21, 16, 21, 16, 11, 16,  8, 25,  7, 16,  8,  3, 17,  8,  4, 11,
         45,  6,  3, 20,  2, 42,  6,  5,  8,  5, 16,  3,  1, 16,  6,  2,  5, 11,
         11, 11,  8,  1, 26, 21, 21,  8,  2,  8, 27, 21,  4,  5,  9, 25, 11, 11,
         19, 14,  5,  8,  8, 11, 25,  2, 11, 45,  8,  4, 21, 44,  6,  4,  3, 14,
         11,  0, 20,  4,  4, 23, 46,  4, 17,  8, 14,  8,  3, 35,  9,  4, 21, 28,
         19, 11,  4,  8, 16,  4, 21, 21,  6, 11, 24, 32, 42, 33, 45, 27, 17, 20,
         26, 16, 39,  8, 35,  8,  8,  9,  4, 44, 19,  8,  3,  8, 16,  8, 33, 11,
         16, 20,  4, 20,  3, 25, 11,  8, 35, 11, 14,  6,  1, 16,  4,  1, 16, 16,
         24,  8, 22, 11, 26,  3, 35,  8,  8,  5, 39,  3, 21, 16, 11, 11,  0, 11,
         33, 11,  6,  5, 37, 26,  3, 17, 35,  3,  8, 33, 35,  8, 46,  8,  6,  4,
         46, 39, 27,  5, 37,  3, 16,  6,  4, 26, 33, 25, 25, 11, 16,  4,  8,  5,
         42, 27,  8, 11, 45, 25,  7, 44, 42,  6, 11, 20,  4,  4,  6,  6,  3,  9,
       

In [7]:
import torchvision.models as models
import torchvision.transforms as transforms

model = models.densenet121(pretrained=True)

num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 48)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/hgurusan/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 114MB/s] 


In [8]:
train_model(model)

100%|██████████| 179/179 [00:16<00:00, 10.78it/s]


Epoch 1/30 - Loss: 3.1838 - Accuracy: 0.1767
Validation Acc: 0.2727


100%|██████████| 179/179 [00:15<00:00, 11.71it/s]


Epoch 2/30 - Loss: 2.0233 - Accuracy: 0.4814
Validation Acc: 0.4318


100%|██████████| 179/179 [00:15<00:00, 11.75it/s]


...

Epoch 28/30 - Loss: 0.0101 - Accuracy: 1.0000
Validation Acc: 0.5559


100%|██████████| 179/179 [00:14<00:00, 12.37it/s]


Epoch 29/30 - Loss: 0.0105 - Accuracy: 0.9998
Validation Acc: 0.5694


100%|██████████| 179/179 [00:14<00:00, 12.41it/s]


Epoch 30/30 - Loss: 0.0101 - Accuracy: 1.0000
Validation Acc: 0.5596


In [9]:
torch.save(model.state_dict(), "DenseNet121-Pretrain")

In [11]:
test_model(model, test_loader, device)

Test Accuracy: 0.5931
MAE: 6.5074
RMSE: 12.6907


(0.5931372549019608,
 tensor([11, 14,  8, 33, 16, 11,  8,  8,  4, 27, 46, 11,  8,  8, 25, 12, 14, 21,
          4, 45, 11, 11,  1, 11,  1,  4,  3, 19,  8,  3,  5,  1, 11,  3,  2,  8,
          8,  1,  7, 19, 21, 13,  8,  6,  8, 11, 35, 33, 35, 33, 16,  4,  8, 21,
         11,  1, 25, 33,  3,  7,  8, 11,  1,  3, 21,  9, 25,  4,  4, 20, 11,  8,
          8,  4, 26,  3, 44,  8,  3, 41,  8,  1, 11, 14, 22, 30, 16,  8,  8,  8,
          8, 16, 14, 11, 39,  1, 26, 33,  6,  3,  6, 42,  8, 17, 45, 11, 16, 20,
         47, 21,  1,  6,  8, 16,  9, 31, 26,  5, 21, 41,  4,  8, 25,  9, 11,  8,
          5, 41,  5,  8, 27,  5,  4,  3,  1, 33, 20, 16,  3, 16,  9, 11,  8,  4,
         16, 37,  1, 21, 16, 16,  3,  8, 14,  9, 11,  8, 35, 21,  4, 11,  2,  8,
         30,  3, 11, 21, 21, 16, 20,  3,  8,  9,  1, 21, 16,  1,  2,  8, 46,  3,
          4, 14,  1, 21,  9, 25, 25, 11,  7, 14,  3,  3, 17,  4,  6, 11, 42, 21,
         25, 39, 35, 11,  3, 45, 21, 39, 21, 21, 33,  8, 34, 33, 25, 14, 26, 11,
       